In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,rank
from pyspark.sql import SparkSession
df_dict_output={}
df_dict={}
read_table_list=[]

In [0]:
for i in dbutils.fs.ls('/mnt/RAW/FILES/GLOBAL_RC_CSA4/DATASETS'):
  df_dict[i.name.split('.')[0].upper().split('_USE')[0]]=spark.read.format('csv').option('inferschema','true').option('header','true').load(i.path)

In [0]:
display(df_dict['PRIMARY_PERSON'].filter(F.col('DEATH_CNT')>0).groupBy('PRSN_GNDR_ID').count())

PRSN_GNDR_ID,count
MALE,182
FEMALE,64


In [0]:
display(df_dict['PRIMARY_PERSON'].filter(F.col('DEATH_CNT')>0).groupBy('CRASH_ID','PRSN_GNDR_ID').count().groupBy('PRSN_GNDR_ID').count())

PRSN_GNDR_ID,count
MALE,180
FEMALE,64


In [0]:
print(df_dict['UNITS'].filter(F.col('VEH_BODY_STYL_ID').like('%MOTORCYCLE%')).count())

784

In [0]:
display(df_dict['PRIMARY_PERSON'].filter(F.col('PRSN_GNDR_ID')=='FEMALE').groupBy('PRSN_GNDR_ID','DRVR_LIC_STATE_ID').count().orderBy(F.col('count').desc(), F.col('PRSN_GNDR_ID')))

PRSN_GNDR_ID,DRVR_LIC_STATE_ID,count
FEMALE,Texas,53319
FEMALE,NA,2793
FEMALE,Unknown,308
FEMALE,Mexico,241
FEMALE,Louisiana,233
FEMALE,New Mexico,233
FEMALE,California,170
FEMALE,Oklahoma,157
FEMALE,Florida,153
FEMALE,Arkansas,115


In [0]:
windowspec=Window.orderBy(F.col('count').desc())
display(df_dict['UNITS'].filter((F.col('TOT_INJRY_CNT')>0) | (F.col('DEATH_CNT') > 0)).groupBy('VEH_MAKE_ID').count().withColumn('RANK', F.dense_rank().over(windowspec)).filter(F.col('RANK').between(5,15)))

VEH_MAKE_ID,count,RANK
NISSAN,2632,5
HONDA,2484,6
GMC,1023,7
HYUNDAI,964,8
KIA,897,9
JEEP,872,10
CHRYSLER,783,11
NA,669,12
MAZDA,602,13
PONTIAC,513,14


In [0]:
windowspec=Window.partitionBy('VEH_BODY_STYL_ID').orderBy(F.col('count').desc())
display(df_dict['PRIMARY_PERSON'].join(df_dict['UNITS'],['CRASH_ID','UNIT_NBR'],'left').select(df_dict['PRIMARY_PERSON']['*'],df_dict['UNITS']['VEH_BODY_STYL_ID']).distinct().groupBy('VEH_BODY_STYL_ID','PRSN_ETHNICITY_ID').count().withColumn('RANK', F.dense_rank().over(windowspec)).filter(F.col('RANK')==1))

VEH_BODY_STYL_ID,PRSN_ETHNICITY_ID,count,RANK
AMBULANCE,WHITE,48,1
BUS,BLACK,87,1
FARM EQUIPMENT,WHITE,21,1
FIRE TRUCK,WHITE,37,1
MOTORCYCLE,WHITE,494,1
NA,WHITE,294,1
NEV-NEIGHBORHOOD ELECTRIC VEHICLE,WHITE,3,1
NOT REPORTED,WHITE,1,1
OTHER (EXPLAIN IN NARRATIVE),WHITE,161,1
"PASSENGER CAR, 2-DOOR",WHITE,4711,1


In [0]:
display(df_dict['PRIMARY_PERSON'].filter(F.col('PRSN_ALC_RSLT_ID')=='Positive').groupBy('DRVR_ZIP').count().orderBy(F.col('count').desc()))

DRVR_ZIP,count
null,169
78521,62
76010,48
79936,42
79938,37
78741,34
79907,34
78550,33
78130,31
79924,31


In [0]:
display(df_dict['PRIMARY_PERSON'].join(df_dict['UNITS'],['CRASH_ID','UNIT_NBR'],'left').select(df_dict['PRIMARY_PERSON']['*'], df_dict['UNITS']['VEH_DMAG_SCL_1_ID']).withColumn('DAMAGE_LEVEL', F.split(F.col('VEH_DMAG_SCL_1_ID'),' ')[1]).join(df_dict['DAMAGES'],'CRASH_ID','left').filter((F.col('DAMAGED_PROPERTY').isNull()) & (F.col('DAMAGE_LEVEL')>4)).select('CRASH_ID').distinct())

CRASH_ID
14852176
14854446
14867150
14910271
14858051
14867846
14874048
14883690
14886011
14893548


In [0]:
print(df_dict['PRIMARY_PERSON'].join(df_dict['UNITS'],['CRASH_ID','UNIT_NBR'],'left').select(df_dict['PRIMARY_PERSON']['*'], df_dict['UNITS']['VEH_DMAG_SCL_1_ID']).withColumn('DAMAGE_LEVEL', F.split(F.col('VEH_DMAG_SCL_1_ID'),' ')[1]).join(df_dict['DAMAGES'],'CRASH_ID','left').filter((F.col('DAMAGED_PROPERTY').isNull()) & (F.col('DAMAGE_LEVEL')>4)).select('CRASH_ID').distinct().count())

9004

In [0]:
windowspec=Window.orderBy(F.col('count').desc())
colour=df_dict['UNITS'].groupBy('VEH_COLOR_ID').count().withColumn('RANK', F.dense_rank().over(windowspec)).filter(F.col('RANK')<=10).select('VEH_COLOR_ID').rdd.flatMap(lambda x:x).collect()
state=df_dict['UNITS'].groupBy('VEH_LIC_STATE_ID').count().withColumn('RANK', F.dense_rank().over(windowspec)).filter(F.col('RANK')<=25).select('VEH_LIC_STATE_ID').rdd.flatMap(lambda x:x).collect()
display(df_dict['UNITS'].filter((F.col('VEH_COLOR_ID').isin(colour)) & (F.col('VEH_LIC_STATE_ID').isin(state)) & ((F.col('CONTRIB_FACTR_2_ID').like('%SPEED%')) | (F.col('CONTRIB_FACTR_1_ID').like('%SPEED%')) | (F.col('CONTRIB_FACTR_P1_ID').like('%SPEED%')) )).join(df_dict['PRIMARY_PERSON'],['CRASH_ID','UNIT_NBR'],'left').select(df_dict['UNITS']['*'], df_dict['PRIMARY_PERSON']['DRVR_LIC_TYPE_ID']).distinct().filter(F.col('DRVR_LIC_TYPE_ID').like('%DRIVER%')).groupBy('VEH_MAKE_ID').count().withColumn('RANK', F.dense_rank().over(windowspec)).filter(F.col('RANK')<=5))

VEH_MAKE_ID,count,RANK
FORD,5213,1
CHEVROLET,4555,2
TOYOTA,2614,3
DODGE,2388,4
HONDA,1669,5
